In [1]:
from tests.parser.unit_tests import get_test_cases, run_tests
tests = get_test_cases()
run_tests(tests)

  Testing PARSE_PRIMARY
-------------------------------------
none
NoneNode(position=(1, 1))
-------------------------------------
5
NumberNode(position=(1, 1), value=5, type=NumberType.INTEGER)
-------------------------------------
4.69
NumberNode(position=(1, 1), value=4.69, type=NumberType.FLOAT)
-------------------------------------
'hello world'
StringNode(position=(1, 1), value='hello world')
-------------------------------------
true
BooleanNode(position=(1, 1), value=True)
-------------------------------------
false
BooleanNode(position=(1, 1), value=False)
-------------------------------------
x
IdentifierNode(position=(1, 1), name='x')

  Testing PARSE_UNARY_OR_PRIMARY
-------------------------------------
some_list[0]
IndexNode(position=(1, 10), collection=IdentifierNode(position=(1, 1), name='some_list'), index=NumberNode(position=(1, 11), value=0, type=NumberType.INTEGER))
-------------------------------------
x[0][1]
IndexNode(position=(1, 5), collection=IndexNode(positio

In [2]:
from src.lexer import Lexer
from src.parser import Parser

def parse(code: str):
    lexer = Lexer(code)
    tokens = lexer.tokenize()
    parser = Parser(tokens)
    return parser.parse()

In [3]:
with open("tests/parser/parse.txt", "r") as f:
    code = f.read()
    
lexer = Lexer(code)
tokens = lexer.tokenize()

parser = Parser(tokens)
ast = parser.parse()
for stmt in ast.statements:
    print(stmt)

NoneNode(position=(2, 1))
NumberNode(position=(3, 1), value=5, type=NumberType.INTEGER)
NumberNode(position=(4, 1), value=4.69, type=NumberType.FLOAT)
StringNode(position=(5, 1), value='hello world')
BooleanNode(position=(6, 1), value=True)
BooleanNode(position=(7, 1), value=False)
BinaryOpNode(position=(12, 1), left=BinaryOpNode(position=(11, 1), left=IdentifierNode(position=(8, 1), name='some_variable'), operator=BinaryOperatorType.MINUS, right=NumberNode(position=(11, 2), value=7, type=NumberType.INTEGER)), operator=BinaryOperatorType.PLUS, right=NumberNode(position=(12, 2), value=4.5, type=NumberType.FLOAT))
UnaryOpNode(position=(13, 1), operator=UnaryOperatorType.NOT, operand=BooleanNode(position=(13, 5), value=True))
IndexNode(position=(16, 10), collection=IdentifierNode(position=(16, 1), name='some_list'), index=NumberNode(position=(16, 11), value=47, type=NumberType.INTEGER))
IndexNode(position=(17, 22), collection=IndexNode(position=(17, 11), collection=IdentifierNode(position

In [4]:
from src.parser.utils import format_ast
test = "BinaryOpNode(left=IdentifierNode(name='index'), operator=BinaryOperatorType.PLUS, right=BinaryOpNode(left=NumberNode(value=7, type=NumberType.INTEGER), operator=BinaryOperatorType.MUL, right=BinaryOpNode(left=BinaryOpNode(left=NumberNode(value=4, type=NumberType.INTEGER), operator=BinaryOperatorType.PLUS, right=BinaryOpNode(left=BinaryOpNode(left=IdentifierNode(name='divisor'), operator=BinaryOperatorType.DIV, right=NumberNode(value=5, type=NumberType.INTEGER)), operator=BinaryOperatorType.MOD, right=NumberNode(value=7, type=NumberType.INTEGER))), operator=BinaryOperatorType.POWER, right=NumberNode(value=2, type=NumberType.INTEGER))))"
ast_str = format_ast(test)
print(ast_str)

BinaryOpNode(
    left=IdentifierNode(
        name='index'
    ),
    operator=BinaryOperatorType.PLUS,
    right=BinaryOpNode(
        left=NumberNode(
            value=7,
            type=NumberType.INTEGER
        ),
        operator=BinaryOperatorType.MUL,
        right=BinaryOpNode(
            left=BinaryOpNode(
                left=NumberNode(
                    value=4,
                    type=NumberType.INTEGER
                ),
                operator=BinaryOperatorType.PLUS,
                right=BinaryOpNode(
                    left=BinaryOpNode(
                        left=IdentifierNode(
                            name='divisor'
                        ),
                        operator=BinaryOperatorType.DIV,
                        right=NumberNode(
                            value=5,
                            type=NumberType.INTEGER
                        )
                    ),
                    operator=BinaryOperatorType.MOD,
                    rig

In [ ]:
code = "function f(a) { return a + 1 }"
ast = parse(code)
print(format_ast(ast))

ProgramNode(
    position=(
        1,
        1
    ),
    statements=[
        FunctionDefNode(
            position=(
                1,
                1
            ),
            name='f',
            parameters=[
                ParameterNode(
                    position=(
                        1,
                        12
                    ),
                    name='a',
                    default_value=None
                )
            ],
            body=[
                ReturnNode(
                    position=(
                        1,
                        17
                    ),
                    value=BinaryOpNode(
                        position=(
                            1,
                            26
                        ),
                        left=IdentifierNode(
                            position=(
                                1,
                                24
                            ),
                            name='a